In [1]:
import pandas as pd
path = '../데이터/2020빅콘테스트 데이터분석분야-퓨쳐스리그_스포츠투아이_제공데이터(.CSV)_시즌별, 시트별 구분/'

In [2]:
team_pitcher_2016 = pd.read_csv(path + '2020빅콘테스트_스포츠투아이_제공데이터_팀투수_2016.csv', encoding='utf-8-sig')
team_pitcher_2017 = pd.read_csv(path + '2020빅콘테스트_스포츠투아이_제공데이터_팀투수_2017.csv', encoding='utf-8-sig')
team_pitcher_2018 = pd.read_csv(path + '2020빅콘테스트_스포츠투아이_제공데이터_팀투수_2018.csv', encoding='utf-8-sig')
team_pitcher_2019 = pd.read_csv(path + '2020빅콘테스트_스포츠투아이_제공데이터_팀투수_2019.csv', encoding='utf-8-sig')
team_pitcher_2020 = pd.read_csv(path + '2020빅콘테스트_스포츠투아이_제공데이터_팀투수_2020.csv', encoding='utf-8-sig')
team_pitcher_list = [team_pitcher_2016, team_pitcher_2017, team_pitcher_2018, team_pitcher_2019, team_pitcher_2020]
team_pitcher = pd.concat(team_pitcher_list)

In [ ]:
for i in team_pitcher_list:
    print(i.info())

In [ ]:
team_pitcher

In [3]:
new_columns = "게임키,일자,팀코드,상대팀코드,더블헤더코드,초말,완투,결과,홀드,이닝*3,투구수,타자,타수,안타,2루타,3루타,홈런,도루,도루실패,희타,희비,4구,고4,사구,삼진,병살타,폭투,보크,실책,실점,자책점,득점권WHIP,2점차이하WHIP,345번타자WHIP"
new_columns = new_columns.split(',')

In [4]:
team_pitcher_2016.columns = new_columns
team_pitcher_2017.columns = new_columns
team_pitcher_2018.columns = new_columns
team_pitcher_2019.columns = new_columns
team_pitcher_2020.columns = new_columns
team_pitcher.columns = new_columns

In [5]:
team_pitcher = team_pitcher[['게임키', '일자', '팀코드', '상대팀코드','결과', '실책', '실점', '자책점']]
team_pitcher

,게임키,일자,팀코드,상대팀코드,결과,실책,실점,자책점
0,20160401HHLG0,20160401,LG,HH,W,0,4,4
1,20160401HHLG0,20160401,HH,LG,L,0,5,4
2,20160401HTNC0,20160401,NC,HT,W,0,4,4
3,20160401HTNC0,20160401,HT,NC,L,0,5,5
4,20160401KTSK0,20160401,SK,KT,L,0,8,8
...,...,...,...,...,...,...,...,...
635,20200719LTSS0,20200719,LT,SS,W,0,1,1
636,20200719OBHT0,20200719,HT,OB,L,0,8,8
637,20200719OBHT0,20200719,OB,HT,W,0,4,4
638,20200719WOSK0,20200719,SK,WO,W,0,3,3


In [6]:
team_pitcher['일자'] = pd.to_datetime(team_pitcher['일자'],format='%Y%m%d', errors='raise')
team_pitcher['시즌'] = team_pitcher['일자'].dt.year

In [7]:
def gainCreate(row):
    row['득점'] = int(team_pitcher[(row['게임키'] == team_pitcher['게임키']) & (team_pitcher['팀코드'] == row['상대팀코드'])]['실점'])
    return row

team_pitcher = team_pitcher.apply(gainCreate, axis=1)

In [ ]:
# 함수 모아놓은 곳
def gainCreate(row):
    row['득점'] = int(team_pitcher[(row['게임키'] == team_pitcher['게임키']) & (team_pitcher['팀코드'] == row['상대팀코드'])]['실점'])
    return row

def createInfo(row):
    previous = team_pitcher[(team_pitcher['팀코드'] == row['팀코드']) & (team_pitcher['일자'] < row['일자']) & (team_pitcher['시즌'] == row['시즌'])].copy()
    if len(previous) == 0:
        row['시작전총득점'] = 0
        row['시작전총실점'] = 0
        row['시작전평균득점'] = 0
        row['시작전평균실점'] = 0
    else:
        gainSum = previous['득점'].sum()
        gainMean = gainSum / len(previous)
        lossSum = previous['실점'].sum()
        lossMean = lossSum / len(previous)
        row['시작전총득점'] = gainSum
        row['시작전총실점'] = lossSum
        row['시작전평균득점'] = gainMean
        row['시작전평균실점'] = lossMean
    return row

def winP(row):
    previous = team_pitcher[(team_pitcher['팀코드'] == row['팀코드']) & (team_pitcher['일자'] < row['일자']) & (team_pitcher['시즌'] == row['시즌'])].copy()
    if len(previous) == 0:
        row['시작전승수'] = 0
        row['시작전패배수'] = 0
        row['시작전승률'] = 0
    else:
        winSum = len(previous[previous['결과'] == "W"])
        lossSum = len(previous[previous['결과'] == "L"])
        winMean = winSum / len(previous)
        row['시작전승수'] = winSum
        row['시작전패배수'] = lossSum
        row['시작전승률'] = winMean
    return row

def verseWinP(row):
    previous = team_pitcher[(team_pitcher['팀코드'] == row['팀코드']) & (team_pitcher['상대팀코드'] == row['상대팀코드']) & (team_pitcher['일자'] < row['일자']) & (team_pitcher['시즌'] == row['시즌'])].copy()
    if len(previous) == 0:
        row['시작전상대승수'] = 0
        row['시작전상대패배수'] = 0
        row['시작전상대승률'] = 0
    else:
        winSum = len(previous[previous['결과'] == "W"])
        lossSum = len(previous[previous['결과'] == "L"])
        winMean = winSum / len(previous)
        row['시작전상대승수'] = winSum
        row['시작전상대패배수'] = lossSum
        row['시작전상대승률'] = winMean
    return row

In [8]:
team_pitcher

,게임키,일자,팀코드,상대팀코드,결과,실책,실점,자책점,시즌,득점
0,20160401HHLG0,2016-04-01,LG,HH,W,0,4,4,2016,5
1,20160401HHLG0,2016-04-01,HH,LG,L,0,5,4,2016,4
2,20160401HTNC0,2016-04-01,NC,HT,W,0,4,4,2016,5
3,20160401HTNC0,2016-04-01,HT,NC,L,0,5,5,2016,4
4,20160401KTSK0,2016-04-01,SK,KT,L,0,8,8,2016,4
...,...,...,...,...,...,...,...,...,...,...
635,20200719LTSS0,2020-07-19,LT,SS,W,0,1,1,2020,2
636,20200719OBHT0,2020-07-19,HT,OB,L,0,8,8,2020,4
637,20200719OBHT0,2020-07-19,OB,HT,W,0,4,4,2020,8
638,20200719WOSK0,2020-07-19,SK,WO,W,0,3,3,2020,4


In [9]:
def createInfo(row):
    previous = team_pitcher[(team_pitcher['팀코드'] == row['팀코드']) & (team_pitcher['일자'] < row['일자']) & (team_pitcher['시즌'] == row['시즌'])].copy()
    if len(previous) == 0:
        row['시작전총득점'] = 0
        row['시작전총실점'] = 0
        row['시작전평균득점'] = 0
        row['시작전평균실점'] = 0
    else:
        gainSum = previous['득점'].sum()
        gainMean = gainSum / len(previous)
        lossSum = previous['실점'].sum()
        lossMean = lossSum / len(previous)
        row['시작전총득점'] = gainSum
        row['시작전총실점'] = lossSum
        row['시작전평균득점'] = gainMean
        row['시작전평균실점'] = lossMean
    return row
team_pitcher = team_pitcher.apply(createInfo, axis=1)

In [10]:
team_pitcher

,게임키,일자,팀코드,상대팀코드,결과,실책,실점,자책점,시즌,득점,시작전총득점,시작전총실점,시작전평균득점,시작전평균실점
0,20160401HHLG0,2016-04-01,LG,HH,W,0,4,4,2016,5,0,0,0.000000,0.000000
1,20160401HHLG0,2016-04-01,HH,LG,L,0,5,4,2016,4,0,0,0.000000,0.000000
2,20160401HTNC0,2016-04-01,NC,HT,W,0,4,4,2016,5,0,0,0.000000,0.000000
3,20160401HTNC0,2016-04-01,HT,NC,L,0,5,5,2016,4,0,0,0.000000,0.000000
4,20160401KTSK0,2016-04-01,SK,KT,L,0,8,8,2016,4,0,0,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
635,20200719LTSS0,2020-07-19,LT,SS,W,0,1,1,2020,2,305,320,5.000000,5.245902
636,20200719OBHT0,2020-07-19,HT,OB,L,0,8,8,2020,4,307,282,5.032787,4.622951
637,20200719OBHT0,2020-07-19,OB,HT,W,0,4,4,2020,8,387,348,6.142857,5.523810
638,20200719WOSK0,2020-07-19,SK,WO,W,0,3,3,2020,4,248,338,3.875000,5.281250


In [11]:
def winP(row):
    previous = team_pitcher[(team_pitcher['팀코드'] == row['팀코드']) & (team_pitcher['일자'] < row['일자']) & (team_pitcher['시즌'] == row['시즌'])].copy()
    if len(previous) == 0:
        row['시작전승수'] = 0
        row['시작전패배수'] = 0
        row['시작전승률'] = 0
    else:
        winSum = len(previous[previous['결과'] == "W"])
        lossSum = len(previous[previous['결과'] == "L"])
        winMean = winSum / len(previous)
        row['시작전승수'] = winSum
        row['시작전패배수'] = lossSum
        row['시작전승률'] = winMean
    return row
team_pitcher = team_pitcher.apply(winP, axis=1)

In [17]:
team_pitcher.head(1443)

,게임키,일자,팀코드,상대팀코드,결과,실책,실점,자책점,시즌,득점,시작전총득점,시작전총실점,시작전평균득점,시작전평균실점,시작전승수,시작전패배수,시작전승률
0,20160401HHLG0,2016-04-01,LG,HH,W,0,4,4,2016,5,0,0,0.000000,0.000000,0,0,0.000000
1,20160401HHLG0,2016-04-01,HH,LG,L,0,5,4,2016,4,0,0,0.000000,0.000000,0,0,0.000000
2,20160401HTNC0,2016-04-01,NC,HT,W,0,4,4,2016,5,0,0,0.000000,0.000000,0,0,0.000000
3,20160401HTNC0,2016-04-01,HT,NC,L,0,5,5,2016,4,0,0,0.000000,0.000000,0,0,0.000000
4,20160401KTSK0,2016-04-01,SK,KT,L,0,8,8,2016,4,0,0,0.000000,0.000000,0,0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1438,20161009WOLT0,2016-10-09,LT,WO,W,0,5,5,2016,8,769,860,5.377622,6.013986,65,78,0.454545
1439,20161009WOLT0,2016-10-09,WO,LT,L,0,8,4,2016,5,808,749,5.650350,5.237762,77,65,0.538462
0,20170331HHOB0,2017-03-31,OB,HH,W,0,0,0,2017,3,0,0,0.000000,0.000000,0,0,0.000000
1,20170331HHOB0,2017-03-31,HH,OB,L,0,3,1,2017,0,0,0,0.000000,0.000000,0,0,0.000000


In [18]:
def verseWinP(row):
    previous = team_pitcher[(team_pitcher['팀코드'] == row['팀코드']) & (team_pitcher['상대팀코드'] == row['상대팀코드']) & (team_pitcher['일자'] < row['일자']) & (team_pitcher['시즌'] == row['시즌'])].copy()
    if len(previous) == 0:
        row['시작전상대승수'] = 0
        row['시작전상대패배수'] = 0
        row['시작전상대승률'] = 0
    else:
        winSum = len(previous[previous['결과'] == "W"])
        lossSum = len(previous[previous['결과'] == "L"])
        winMean = winSum / len(previous)
        row['시작전상대승수'] = winSum
        row['시작전상대패배수'] = lossSum
        row['시작전상대승률'] = winMean
    return row
team_pitcher = team_pitcher.apply(verseWinP, axis=1)

In [81]:
team_pitcher.columns

Index(['게임키', '일자', '팀코드', '상대팀코드', '결과', '실책', '실점', '자책점', '시즌', '득점',
       '시작전총득점', '시작전총실점', '시작전평균득점', '시작전평균실점', '시작전승수', '시작전패배수', '시작전승률',
       '시작전상대승수', '시작전상대패배수', '시작전상대승률'],
      dtype='object')

In [23]:
team_pitcher.to_csv("../EDA결과/EDA2-경기전 팀 스탯.csv", encoding='utf-8-sig')

In [45]:
team_stat_2016 = team_pitcher[team_pitcher['시즌'] == 2016]
team_stat_2017 = team_pitcher[team_pitcher['시즌'] == 2017]
team_stat_2018 = team_pitcher[team_pitcher['시즌'] == 2018]
team_stat_2019 = team_pitcher[team_pitcher['시즌'] == 2019]
team_stat_2020 = team_pitcher[team_pitcher['시즌'] == 2020]
team_stat_list = [team_stat_2016, team_stat_2017, team_stat_2018, team_stat_2019, team_stat_2020]

In [46]:
for team in team_stat_list: 
    team['정석피타고리안'] = (team['시작전총득점'] ** 2) / ((team['시작전총득점'] ** 2) + (team['시작전총실점'] ** 2))
    team['논문용피타고리안'] = team['시작전총득점'] / (team['시작전총득점'] + team['시작전총실점'])

C:\Users\Yang\Anaconda3\Lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\Yang\Anaconda3\Lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [47]:
team_stat_2018

,게임키,일자,팀코드,상대팀코드,결과,실책,실점,자책점,시즌,득점,...,시작전평균득점,시작전평균실점,시작전승수,시작전패배수,시작전승률,시작전상대승수,시작전상대패배수,시작전상대승률,정석피타고리안,논문용피타고리안
0,20180324HHWO0,2018-03-24,WO,HH,W,1,3,2,2018,6,...,0.000000,0.000000,0,0,0.000000,0,0,0.000000,NaN,NaN
1,20180324HHWO0,2018-03-24,HH,WO,L,0,6,5,2018,3,...,0.000000,0.000000,0,0,0.000000,0,0,0.000000,NaN,NaN
2,20180324KTHT0,2018-03-24,HT,KT,L,0,5,5,2018,4,...,0.000000,0.000000,0,0,0.000000,0,0,0.000000,NaN,NaN
3,20180324KTHT0,2018-03-24,KT,HT,W,0,4,4,2018,5,...,0.000000,0.000000,0,0,0.000000,0,0,0.000000,NaN,NaN
4,20180324LGNC0,2018-03-24,NC,LG,W,0,2,2,2018,4,...,0.000000,0.000000,0,0,0.000000,0,0,0.000000,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1435,20181013NCHH0,2018-10-13,NC,HH,L,0,10,10,2018,8,...,4.559441,5.832168,58,84,0.405594,7,8,0.466667,0.379334,0.438762
1436,20181013WOSS0,2018-10-13,SS,WO,W,0,5,5,2018,12,...,5.342657,5.573427,67,72,0.468531,4,11,0.266667,0.478869,0.489430
1437,20181013WOSS0,2018-10-13,WO,SS,L,1,12,10,2018,5,...,5.734266,5.370629,75,68,0.524476,11,4,0.733333,0.532711,0.516373
1438,20181014OBLT0,2018-10-14,LT,OB,L,0,5,5,2018,1,...,5.734266,5.881119,68,73,0.475524,3,12,0.200000,0.487359,0.493679


In [82]:
team_stat_2016 = team_stat_2016.fillna(0)
team_stat_2017 = team_stat_2017.fillna(0)
team_stat_2018 = team_stat_2018.fillna(0)
team_stat_2019 = team_stat_2019.fillna(0)
team_stat_2020 = team_stat_2020.fillna(0)

In [83]:
team_stat_2020

,게임키,일자,팀코드,상대팀코드,결과,실책,실점,자책점,시즌,득점,...,시작전평균득점,시작전평균실점,시작전승수,시작전패배수,시작전승률,시작전상대승수,시작전상대패배수,시작전상대승률,정석피타고리안,논문용피타고리안
0,20200505HHSK0,2020-05-05,SK,HH,L,0,3,3,2020,0,...,0.000000,0.000000,0,0,0.000000,0,0,0.00,0.000000,0.000000
1,20200505HHSK0,2020-05-05,HH,SK,W,0,0,0,2020,3,...,0.000000,0.000000,0,0,0.000000,0,0,0.00,0.000000,0.000000
2,20200505LTKT0,2020-05-05,KT,LT,L,0,7,7,2020,2,...,0.000000,0.000000,0,0,0.000000,0,0,0.00,0.000000,0.000000
3,20200505LTKT0,2020-05-05,LT,KT,W,0,2,2,2020,7,...,0.000000,0.000000,0,0,0.000000,0,0,0.00,0.000000,0.000000
4,20200505NCSS0,2020-05-05,SS,NC,L,0,4,4,2020,0,...,0.000000,0.000000,0,0,0.000000,0,0,0.00,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
635,20200719LTSS0,2020-07-19,LT,SS,W,0,1,1,2020,2,...,5.000000,5.245902,29,32,0.475410,2,6,0.25,0.476014,0.488000
636,20200719OBHT0,2020-07-19,HT,OB,L,0,8,8,2020,4,...,5.032787,4.622951,33,28,0.540984,2,6,0.25,0.542368,0.521222
637,20200719OBHT0,2020-07-19,OB,HT,W,0,4,4,2020,8,...,6.142857,5.523810,37,26,0.587302,6,2,0.75,0.552912,0.526531
638,20200719WOSK0,2020-07-19,SK,WO,W,0,3,3,2020,4,...,3.875000,5.281250,20,44,0.312500,2,6,0.25,0.349956,0.423208


In [84]:
import plotly.graph_objects as go
team_code =['NC','OB','WO','HT','LG','SS','KT','LT','SK','HH']

year=2016
for team_season in team_stat_list:
    for code in team_code:
        stat = team_season[team_season['팀코드'] == code]
        fig = go.Figure()
        fig.add_trace(
            go.Scatter(
                x=stat['일자'], y=stat['시작전승률'], mode='lines', name='승률'
            )
        )
        fig.add_trace(
            go.Scatter(
                x=stat['일자'], y=stat['정석피타고리안'], mode='lines+markers+text', name='정석피타'
            )
        )
        fig.add_trace(
            go.Scatter(
                x=stat['일자'], y=stat['논문용피타고리안'], mode='lines+markers+text', name='논문피타'
            )
        )
        fig.update_layout(
        {
            "title": {
                "text": "{}년 {} 승률, 피타고리안 기대승률 비교".format(year,code),
                "x": 0.5,
                "y": 0.9,
                "font": {
                    "size": 20
                }
            },
            "showlegend": True,
            "xaxis": {
                "title": "일자",
                "showticklabels":True,

            },
            "yaxis": {
                "title": "승률"
            },

            }
        )


        fig.show()
    year+=1

In [66]:
def pitagoriPred(row):
    game = team_stat_2016[team_stat_2016['게임키'] == row['게임키']]
    verse = game[row['상대팀코드'] == game['팀코드']]
    if float(row['논문용피타고리안']) > float(verse['논문용피타고리안']):
        row['논문피타고리안승패예측'] = 'W'
    elif float(row['논문용피타고리안']) == float(verse['논문용피타고리안']):
        row['논문피타고리안승패예측'] = 'D'
    else:
        row['논문피타고리안승패예측'] = 'L'
    return row
team_stat_2016 = team_stat_2016.apply(pitagoriPred, axis=1)

In [67]:
team_stat_2016

,게임키,일자,팀코드,상대팀코드,결과,실책,실점,자책점,시즌,득점,...,시작전평균실점,시작전승수,시작전패배수,시작전승률,시작전상대승수,시작전상대패배수,시작전상대승률,정석피타고리안,논문용피타고리안,논문피타고리안승패예측
0,20160401HHLG0,2016-04-01,LG,HH,W,0,4,4,2016,5,...,0.000000,0,0,0.000000,0,0,0.000000,0.000000,0.000000,D
1,20160401HHLG0,2016-04-01,HH,LG,L,0,5,4,2016,4,...,0.000000,0,0,0.000000,0,0,0.000000,0.000000,0.000000,D
2,20160401HTNC0,2016-04-01,NC,HT,W,0,4,4,2016,5,...,0.000000,0,0,0.000000,0,0,0.000000,0.000000,0.000000,D
3,20160401HTNC0,2016-04-01,HT,NC,L,0,5,5,2016,4,...,0.000000,0,0,0.000000,0,0,0.000000,0.000000,0.000000,D
4,20160401KTSK0,2016-04-01,SK,KT,L,0,8,8,2016,4,...,0.000000,0,0,0.000000,0,0,0.000000,0.000000,0.000000,D
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1435,20161008SSSK0,2016-10-08,SS,SK,L,0,7,6,2016,6,...,6.027972,65,77,0.454545,8,7,0.533333,0.490633,0.495316,W
1436,20161009KTNC0,2016-10-09,NC,KT,L,0,7,7,2016,4,...,4.776224,83,57,0.580420,9,5,0.600000,0.609338,0.555339,W
1437,20161009KTNC0,2016-10-09,KT,NC,W,1,4,3,2016,7,...,6.454545,52,89,0.363636,5,9,0.333333,0.341710,0.418766,L
1438,20161009WOLT0,2016-10-09,LT,WO,W,0,5,5,2016,8,...,6.013986,65,78,0.454545,6,9,0.400000,0.444311,0.472069,L


In [73]:
len(team_stat_2016[(team_stat_2016['결과'] == 'W') & (team_stat_2016['결과'] == team_stat_2016['논문피타고리안승패예측'])]) / len(team_stat_2016[(team_stat_2016['결과'] == 'W')])

0.5329593267882188

In [72]:
len(team_stat_2016[(team_stat_2016['결과'] == 'W')])

713

In [85]:
# 상대 전적을 통해 전적이 높은 놈이 얼마나 많이 이기는지 보기
def verseWinPred(row):
    game = team_stat_2016[team_stat_2016['게임키'] == row['게임키']]
    verse = game[row['상대팀코드'] == game['팀코드']]
    if float(row['시작전상대승률']) > float(verse['시작전상대승률']):
        row['시작전상대승률예측'] = 'W'
    elif float(row['시작전상대승률']) == float(verse['시작전상대승률']):
        row['시작전상대승률예측'] = 'D'
    else:
        row['시작전상대승률예측'] = 'L'
    return row
team_stat_2016 = team_stat_2016.apply(verseWinPred, axis=1)

In [87]:
team_stat_2016.tail(20)

,게임키,일자,팀코드,상대팀코드,결과,실책,실점,자책점,시즌,득점,...,시작전승수,시작전패배수,시작전승률,시작전상대승수,시작전상대패배수,시작전상대승률,정석피타고리안,논문용피타고리안,논문피타고리안승패예측,시작전상대승률예측
1420,20161006LGLT0,2016-10-06,LT,LG,L,0,4,4,2016,1,...,64,77,0.453901,8,7,0.533333,0.445058,0.472446,L,W
1421,20161006LGLT0,2016-10-06,LG,LT,W,0,1,1,2016,4,...,70,70,0.492958,7,8,0.466667,0.491117,0.495558,W,L
1422,20161006SKNC0,2016-10-06,NC,SK,W,0,2,1,2016,4,...,82,56,0.581560,9,6,0.600000,0.611584,0.556505,W,W
1423,20161006SKNC0,2016-10-06,SK,NC,L,0,4,4,2016,2,...,68,74,0.478873,6,9,0.400000,0.480245,0.490119,L,L
1424,20161006SSHT0,2016-10-06,HT,SS,L,0,4,4,2016,3,...,70,71,0.492958,8,7,0.533333,0.512737,0.506369,W,W
1425,20161006SSHT0,2016-10-06,SS,HT,W,0,3,1,2016,4,...,64,77,0.450704,7,8,0.466667,0.490007,0.495003,L,L
1426,20161007KTNC0,2016-10-07,NC,KT,L,0,6,5,2016,2,...,83,56,0.584507,9,4,0.642857,0.612417,0.556937,W,W
1427,20161007KTNC0,2016-10-07,KT,NC,W,0,2,2,2016,6,...,51,89,0.359155,4,9,0.285714,0.338615,0.417089,L,L
1428,20161007WOLT0,2016-10-07,LT,WO,W,0,4,4,2016,5,...,64,78,0.450704,5,9,0.357143,0.443392,0.471605,L,L
1429,20161007WOLT0,2016-10-07,WO,LT,L,0,5,5,2016,4,...,77,64,0.542254,9,5,0.642857,0.538702,0.519380,W,W


In [88]:
len(team_stat_2016[(team_stat_2016['결과'] == 'W') & (team_stat_2016['결과'] == team_stat_2016['시작전상대승률예측'])]) / len(team_stat_2016[(team_stat_2016['결과'] == 'W')])

0.3955119214586255